In [1]:
import nltk
import string
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
%matplotlib inline
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv("Sentiment.csv")

In [3]:
df

,id,candidate,candidate_confidence,relevant_yn,relevant_yn_confidence,sentiment,sentiment_confidence,subject_matter,subject_matter_confidence,candidate_gold,...,relevant_yn_gold,retweet_count,sentiment_gold,subject_matter_gold,text,tweet_coord,tweet_created,tweet_id,tweet_location,user_timezone
0,1,No candidate mentioned,1.0000,yes,1.0000,Neutral,0.6578,None of the above,1.0000,NaN,...,NaN,5,NaN,NaN,RT @NancyLeeGrahn: How did everyone feel about...,NaN,2015-08-07 09:54:46 -0700,629697200650592256,NaN,Quito
1,2,Scott Walker,1.0000,yes,1.0000,Positive,0.6333,None of the above,1.0000,NaN,...,NaN,26,NaN,NaN,RT @ScottWalker: Didn't catch the full #GOPdeb...,NaN,2015-08-07 09:54:46 -0700,629697199560069120,NaN,NaN
2,3,No candidate mentioned,1.0000,yes,1.0000,Neutral,0.6629,None of the above,0.6629,NaN,...,NaN,27,NaN,NaN,RT @TJMShow: No mention of Tamir Rice and the ...,NaN,2015-08-07 09:54:46 -0700,629697199312482304,NaN,NaN
3,4,No candidate mentioned,1.0000,yes,1.0000,Positive,1.0000,None of the above,0.7039,NaN,...,NaN,138,NaN,NaN,RT @RobGeorge: That Carly Fiorina is trending ...,NaN,2015-08-07 09:54:45 -0700,629697197118861312,Texas,Central Time (US & Canada)
4,5,Donald Trump,1.0000,yes,1.0000,Positive,0.7045,None of the above,1.0000,NaN,...,NaN,156,NaN,NaN,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,NaN,2015-08-07 09:54:45 -0700,629697196967903232,NaN,Arizona
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13866,13867,No candidate mentioned,1.0000,yes,1.0000,Negative,0.7991,Abortion,0.6014,No candidate mentioned,...,yes,7,Negative,Abortion\nWomen's Issues (not abortion though),RT @cappy_yarbrough: Love to see men who will ...,NaN,2015-08-07 09:29:43 -0700,629690895479250944,Como,NaN
13867,13868,Mike Huckabee,0.9611,yes,1.0000,Positive,0.7302,None of the above,0.9229,Mike Huckabee,...,yes,1,NaN,NaN,RT @georgehenryw: Who thought Huckabee exceede...,NaN,2015-08-07 09:25:02 -0700,629689719056568320,USA,NaN
13868,13869,Ted Cruz,1.0000,yes,1.0000,Positive,0.8051,None of the above,0.9647,Ted Cruz,...,yes,67,Positive\nNeutral,NaN,"RT @Lrihendry: #TedCruz As President, I will a...",NaN,2015-08-07 07:19:18 -0700,629658075784282112,NaN,NaN
13869,13870,Donald Trump,1.0000,yes,1.0000,Negative,1.0000,Women's Issues (not abortion though),0.9202,Donald Trump,...,yes,149,NaN,Women's Issues (not abortion though),RT @JRehling: #GOPDebate Donald Trump says tha...,NaN,2015-08-07 09:54:04 -0700,629697023663546368,NaN,NaN


In [4]:
df.shape

(13871, 21)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13871 entries, 0 to 13870
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         13871 non-null  int64  
 1   candidate                  13775 non-null  object 
 2   candidate_confidence       13871 non-null  float64
 3   relevant_yn                13871 non-null  object 
 4   relevant_yn_confidence     13871 non-null  float64
 5   sentiment                  13871 non-null  object 
 6   sentiment_confidence       13871 non-null  float64
 7   subject_matter             13545 non-null  object 
 8   subject_matter_confidence  13871 non-null  float64
 9   candidate_gold             28 non-null     object 
 10  name                       13871 non-null  object 
 11  relevant_yn_gold           32 non-null     object 
 12  retweet_count              13871 non-null  int64  
 13  sentiment_gold             15 non-null     obj

In [6]:
df.isnull().sum()

id                               0
candidate                       96
candidate_confidence             0
relevant_yn                      0
relevant_yn_confidence           0
sentiment                        0
sentiment_confidence             0
subject_matter                 326
subject_matter_confidence        0
candidate_gold               13843
name                             0
relevant_yn_gold             13839
retweet_count                    0
sentiment_gold               13856
subject_matter_gold          13853
text                             0
tweet_coord                  13850
tweet_created                    0
tweet_id                         0
tweet_location                3912
user_timezone                 4403
dtype: int64

In [7]:
df.drop(columns=["candidate_gold","relevant_yn_gold","sentiment_gold","subject_matter_gold","tweet_coord","id"], inplace=True)

In [8]:
df.head()

,candidate,candidate_confidence,relevant_yn,relevant_yn_confidence,sentiment,sentiment_confidence,subject_matter,subject_matter_confidence,name,retweet_count,text,tweet_created,tweet_id,tweet_location,user_timezone
0,No candidate mentioned,1.0,yes,1.0,Neutral,0.6578,None of the above,1.0000,I_Am_Kenzi,5,RT @NancyLeeGrahn: How did everyone feel about...,2015-08-07 09:54:46 -0700,629697200650592256,NaN,Quito
1,Scott Walker,1.0,yes,1.0,Positive,0.6333,None of the above,1.0000,PeacefulQuest,26,RT @ScottWalker: Didn't catch the full #GOPdeb...,2015-08-07 09:54:46 -0700,629697199560069120,NaN,NaN
2,No candidate mentioned,1.0,yes,1.0,Neutral,0.6629,None of the above,0.6629,PussssyCroook,27,RT @TJMShow: No mention of Tamir Rice and the ...,2015-08-07 09:54:46 -0700,629697199312482304,NaN,NaN
3,No candidate mentioned,1.0,yes,1.0,Positive,1.0000,None of the above,0.7039,MattFromTexas31,138,RT @RobGeorge: That Carly Fiorina is trending ...,2015-08-07 09:54:45 -0700,629697197118861312,Texas,Central Time (US & Canada)
4,Donald Trump,1.0,yes,1.0,Positive,0.7045,None of the above,1.0000,sharonDay5,156,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,2015-08-07 09:54:45 -0700,629697196967903232,NaN,Arizona


In [9]:
df.isnull().sum()

candidate                      96
candidate_confidence            0
relevant_yn                     0
relevant_yn_confidence          0
sentiment                       0
sentiment_confidence            0
subject_matter                326
subject_matter_confidence       0
name                            0
retweet_count                   0
text                            0
tweet_created                   0
tweet_id                        0
tweet_location               3912
user_timezone                4403
dtype: int64

In [10]:
# removes pattern in the imput text
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for word in r:
        input_txt = re.sub(word, "",input_txt)
    return input_txt

In [11]:
#remove twitter handels (@user)
df['clean_tweet'] = np.vectorize(remove_pattern)(df['text'],"@[\w]*")
df.head()

,candidate,candidate_confidence,relevant_yn,relevant_yn_confidence,sentiment,sentiment_confidence,subject_matter,subject_matter_confidence,name,retweet_count,text,tweet_created,tweet_id,tweet_location,user_timezone,clean_tweet
0,No candidate mentioned,1.0,yes,1.0,Neutral,0.6578,None of the above,1.0000,I_Am_Kenzi,5,RT @NancyLeeGrahn: How did everyone feel about...,2015-08-07 09:54:46 -0700,629697200650592256,NaN,Quito,RT : How did everyone feel about the Climate C...
1,Scott Walker,1.0,yes,1.0,Positive,0.6333,None of the above,1.0000,PeacefulQuest,26,RT @ScottWalker: Didn't catch the full #GOPdeb...,2015-08-07 09:54:46 -0700,629697199560069120,NaN,NaN,RT : Didn't catch the full #GOPdebate last nig...
2,No candidate mentioned,1.0,yes,1.0,Neutral,0.6629,None of the above,0.6629,PussssyCroook,27,RT @TJMShow: No mention of Tamir Rice and the ...,2015-08-07 09:54:46 -0700,629697199312482304,NaN,NaN,RT : No mention of Tamir Rice and the #GOPDeba...
3,No candidate mentioned,1.0,yes,1.0,Positive,1.0000,None of the above,0.7039,MattFromTexas31,138,RT @RobGeorge: That Carly Fiorina is trending ...,2015-08-07 09:54:45 -0700,629697197118861312,Texas,Central Time (US & Canada),RT : That Carly Fiorina is trending -- hours a...
4,Donald Trump,1.0,yes,1.0,Positive,0.7045,None of the above,1.0000,sharonDay5,156,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,2015-08-07 09:54:45 -0700,629697196967903232,NaN,Arizona,RT : #GOPDebate w/ delivered the highest rati...


In [12]:
df['sentiment'].unique()

array(['Neutral', 'Positive', 'Negative'], dtype=object)

In [13]:
# extract the hashtag
def hashtag_extract(tweets):
    hashtags = []
    # loop words in the tweet
    for tweet in df['text']:
        ht = re.findall(r"#(\w+)", tweet)
        hashtags.append(ht)
    return hashtags

In [14]:
ht_positive = hashtag_extract(df['clean_tweet'][df['sentiment']=='Positive'])

ht_neutral = hashtag_extract(df['clean_tweet'][df['sentiment']=='Neutral'])

ht_negative = hashtag_extract(df['clean_tweet'][df['sentiment']=='Negative'])

In [15]:
ht_positive = sum(ht_positive, [])
ht_negative = sum(ht_negative, [])
ht_neutral = sum(ht_neutral, [])

In [16]:
freq = nltk.FreqDist(ht_positive)
d = pd.DataFrame({'Hashtag': list(freq.keys()),
                 'Count': list(freq.values())})
d.head()

,Hashtag,Count
0,GOPDebate,8373
1,GOPdebate,441
2,Walker16,26
3,Trump2016,72
4,MSNBC,4


In [17]:
d.shape

(2150, 2)

In [18]:
df['clean_tweet']

0        RT : How did everyone feel about the Climate C...
1        RT : Didn't catch the full #GOPdebate last nig...
2        RT : No mention of Tamir Rice and the #GOPDeba...
3        RT : That Carly Fiorina is trending -- hours a...
4        RT : #GOPDebate w/  delivered the highest rati...
                               ...                        
13866    RT : Love to see men who will never be faced w...
13867    RT : Who thought Huckabee exceeded their expec...
13868    RT : #TedCruz As President, I will always tell...
13869    RT : #GOPDebate Donald Trump says that he does...
13870    RT : #TedCruz headed into the Presidential Deb...
Name: clean_tweet, Length: 13871, dtype: object

In [19]:
df['clean_tweet'] = df['clean_tweet'].str.replace('RT : ', '')

In [20]:
x = df['clean_tweet']

In [21]:
# converting the strings into lowercase
x = x.apply(lambda x: x.lower())
print(x)

0        how did everyone feel about the climate change...
1        didn't catch the full #gopdebate last night. h...
2        no mention of tamir rice and the #gopdebate wa...
3        that carly fiorina is trending -- hours after ...
4        #gopdebate w/  delivered the highest ratings i...
                               ...                        
13866    love to see men who will never be faced with a...
13867    who thought huckabee exceeded their expectatio...
13868    #tedcruz as president, i will always tell the ...
13869    #gopdebate donald trump says that he doesn't h...
13870    #tedcruz headed into the presidential debates....
Name: clean_tweet, Length: 13871, dtype: object


In [22]:
# converting numbers to text (eg: 3 -> "three")
import inflect
q = inflect.engine()

def convert_num(text):
    temp_string = text.split()
    new_str=[]

    for word in temp_string:
        if word.isdigit():
            temp=q.number_to_words(word)
            new_str.append(temp)

        else:
            new_str.append(word)

    temp_str = ' '.join(new_str)
    return temp_str

In [23]:
x= x.apply(convert_num)
print(x)

0        how did everyone feel about the climate change...
1        didn't catch the full #gopdebate last night. h...
2        no mention of tamir rice and the #gopdebate wa...
3        that carly fiorina is trending -- hours after ...
4        #gopdebate w/ delivered the highest ratings in...
                               ...                        
13866    love to see men who will never be faced with a...
13867    who thought huckabee exceeded their expectatio...
13868    #tedcruz as president, i will always tell the ...
13869    #gopdebate donald trump says that he doesn't h...
13870    #tedcruz headed into the presidential debates....
Name: clean_tweet, Length: 13871, dtype: object


In [24]:
def rem_pun(text):
    translator = str.maketrans('','', string.punctuation)
    return text.translate(translator)
x = x.apply(rem_pun)

In [25]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download("stopwords")
nltk.download("punkt")

def remove_stopwords(text):
    stop_words = set(stopwords.words("english"))
    word_tokens = word_tokenize(text)
    filtered_text = [word for word in word_tokens if word not in stop_words]
    return ' '.join(filtered_text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [26]:
x=x.apply(remove_stopwords)
x

0        everyone feel climate change question last nig...
1        didnt catch full gopdebate last night scotts b...
2          mention tamir rice gopdebate held cleveland wow
3        carly fiorina trending hours debate men justco...
4        gopdebate w delivered highest ratings history ...
                               ...                        
13866    love see men never faced pregnancy talk body ❤...
13867    thought huckabee exceeded expectations gopdeba...
13868    tedcruz president always tell truth said would...
13869    gopdebate donald trump says doesnt time politi...
13870    tedcruz headed presidential debates go ted gop...
Name: clean_tweet, Length: 13871, dtype: object

In [27]:
from nltk.stem import wordnet
from nltk.tokenize import word_tokenize
lemma = wordnet.WordNetLemmatizer()
nltk.download('wordnet')

def lemmatize_word(text):
    word_tokens = word_tokenize(text)
    lemmas = [lemma.lemmatize(word, pos='v') for word in word_tokens]
    return ' '.join(lemmas)

x=x.apply(lemmatize_word)
x

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


0        everyone feel climate change question last nig...
1        didnt catch full gopdebate last night scotts b...
2          mention tamir rice gopdebate hold cleveland wow
3        carly fiorina trend hours debate men justcompl...
4        gopdebate w deliver highest rat history presid...
                               ...                        
13866    love see men never face pregnancy talk body ❤️...
13867    think huckabee exceed expectations gopdebate i...
13868    tedcruz president always tell truth say would ...
13869    gopdebate donald trump say doesnt time politic...
13870    tedcruz head presidential debate go ted gopdeb...
Name: clean_tweet, Length: 13871, dtype: object

In [28]:
import nltk

# Download the vader_lexicon resource
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [29]:
from nltk.sentiment import SentimentIntensityAnalyzer

sia = SentimentIntensityAnalyzer()
x['Polarity_Score'] = x.apply(lambda x: sia.polarity_scores(x))

In [30]:
x['Polarity_Score']

0        {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...
1        {'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'comp...
2        {'neg': 0.0, 'neu': 0.612, 'pos': 0.388, 'comp...
3        {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...
4        {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...
                               ...                        
13866    {'neg': 0.0, 'neu': 0.682, 'pos': 0.318, 'comp...
13867    {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...
13868    {'neg': 0.0, 'neu': 0.753, 'pos': 0.247, 'comp...
13869    {'neg': 0.0, 'neu': 0.789, 'pos': 0.211, 'comp...
13870    {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...
Name: clean_tweet, Length: 13871, dtype: object

In [31]:
df['sentiment'].value_counts()

Negative    8493
Neutral     3142
Positive    2236
Name: sentiment, dtype: int64

In [32]:
from sklearn.model_selection import train_test_split
x = df['clean_tweet']
y = df['sentiment']

# Split the data into train and test sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [33]:
num_words = len(x)
print(num_words)

13871


In [34]:
import nltk
nltk.download('punkt')

from nltk.tokenize import word_tokenize
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.preprocessing import LabelEncoder

import warnings
warnings.filterwarnings('ignore')
sns.set()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [35]:
tokenizer = Tokenizer(num_words)
tokenizer.fit_on_texts(x_train)
x_train = tokenizer.texts_to_sequences(x_train)
x_train = pad_sequences(x_train, maxlen=36, truncating='post', padding='post')

In [36]:
x_train[1], len(x_train[1])

(array([ 12, 377, 626,  24,  63,   7, 655, 291, 591,   1,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0], dtype=int32),
 36)

In [37]:
x_test = tokenizer.texts_to_sequences(x_test)
x_test = pad_sequences(x_test, maxlen=36, truncating='post', padding='post')

In [38]:
x_test[0], len(x_test[0])

(array([  4,  36,   2, 191, 266,   7,  31,  52,   1,  24, 484,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0], dtype=int32),
 36)

In [39]:
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(11096, 36) (11096,)
(2775, 36) (2775,)


In [40]:
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.fit_transform(y_test)

In [41]:
model = Sequential()

model.add(Embedding(input_dim=num_words, output_dim=100, input_length=36, trainable=True))
model.add(LSTM(100, dropout=0.1, return_sequences=True))
model.add(LSTM(100, dropout=0.1))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [42]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 36, 100)           1387100   
                                                                 
 lstm (LSTM)                 (None, 36, 100)           80400     
                                                                 
 lstm_1 (LSTM)               (None, 100)               80400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 1,548,001
Trainable params: 1,548,001
Non-trainable params: 0
_________________________________________________________________


In [49]:
history = model.fit(x_train, y_train, epochs=5, batch_size=128, validation_data=(x_test, y_test))

Epoch 1/5
87/87 [==============================] - 27s 304ms/step - loss: -20.6092 - accuracy: 0.7176 - val_loss: -4.4688 - val_accuracy: 0.6083
Epoch 2/5
87/87 [==============================] - 28s 319ms/step - loss: -22.9947 - accuracy: 0.7068 - val_loss: -4.8134 - val_accuracy: 0.6169
Epoch 3/5
87/87 [==============================] - 27s 309ms/step - loss: -23.8878 - accuracy: 0.6909 - val_loss: -4.2246 - val_accuracy: 0.5820
Epoch 4/5
87/87 [==============================] - 26s 302ms/step - loss: -27.0141 - accuracy: 0.7266 - val_loss: -4.5024 - val_accuracy: 0.6065
Epoch 5/5
87/87 [==============================] - 40s 458ms/step - loss: -29.4355 - accuracy: 0.7213 - val_loss: -3.0530 - val_accuracy: 0.5953


In [50]:
# Evaluate the model
loss, accuracy = model.evaluate(x_test, y_test)

print("Accuracy:", accuracy)

87/87 [==============================] - 2s 27ms/step - loss: -3.0530 - accuracy: 0.5953
Accuracy: 0.5953153371810913
